# L4: Ferramentas para uma campanha de divulgação do cliente

Nesta lição, você aprenderá mais sobre Ferramentas. Você se concentrará em três elementos principais das Ferramentas:
- Versatilidade
- Tolerância ao erro
- Cache

As bibliotecas já estão instaladas nas salas de aula. Se você estiver executando este notebook em sua própria máquina, poderá instalar o seguinte:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
from crewai import Agent, Task, Crew

**Observação**:
- O vídeo usa `gpt-4-turbo`, mas devido a certas restrições, e para oferecer este curso gratuitamente a todos, o código que você executará aqui usará `gpt-3.5-turbo`.
- Você pode usar `gpt-4-turbo` ao executar o notebook _localmente_ (usar `gpt-4-turbo` não funcionará na plataforma)
- Obrigado pela sua compreensão!

In [3]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Criando Agents

In [4]:
sales_rep_agent = Agent(
    role="Representante de Vendas",
    goal="Identifique leads de alto valor que correspondam a "
          "nosso perfil de cliente ideal",
    backstory=(
         "Como parte da equipe dinâmica de vendas da Inovabot, "
         "sua missão é vasculhar"
         "o cenário digital para leads em potencial."
         "Armado com ferramentas de última geração"
         "e uma mentalidade estratégica, você analisa dados"
         "tendências e interações para"
         "descobrir oportunidades que outros podem ignorar."
         "Seu trabalho é crucial para pavimentar o caminho"
         "para compromissos significativos e impulsionando o crescimento da empresa."
    ),
    allow_delegation=False,
    verbose=True
)

In [5]:
lead_sales_rep_agent = Agent (
     role="Representante Líder de Vendas",
     goal="Nutrir leads com comunicações personalizadas e atraentes e responder em português do Brasil",
     backstory=(
         "Dentro do vibrante ecossistema do departamento de vendas da CrewAI"
         "você se destaca como a ponte entre potenciais clientes"
         "e as soluções que eles precisam."
         "Ao criar mensagens envolventes e personalizadas, "
         "você não apenas informa os leads sobre nossas ofertas"
         "mas também faça com que se sintam vistos e ouvidos."
         "Seu papel é fundamental na conversão de juros"
         "em ação, guiando leads ao longo da jornada"
         "da curiosidade ao compromisso."
     ),
     allow_delegation=False,
     verbose=True
)

## Criando Ferramentas

### crewAI Tools

In [6]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [7]:
directory_read_tool = DirectoryReadTool(directory='./instrucoes')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Ferramenta personalizada

- Crie uma ferramenta personalizada usando o CrewAi [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [8]:
from crewai_tools import BaseTool

- Toda ferramenta precisa ter um `nome` e uma `descrição`.
- Para simplificar e para fins de sala de aula, `SentimentAnalysisTool` retornará `positivo` para cada texto.
- Ao executar localmente, você pode personalizar o código com sua lógica na função `_run`.

In [9]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Ferramenta de análise de sentimento"
    description: str = ("Analisa o sentimento do texto " 
                        "para garantir uma comunicação positiva e envolvente.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [10]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Criando Tarefas 
- A tarefa de criação de perfil de lead está usando ferramentas CrewAI.

In [11]:
lead_profiling_task = Task(
    description=(
        "Conduza uma análise aprofundada de {lead_name},"
         "uma empresa do setor {industry}"
         "que recentemente demonstrou interesse em nossas soluções."
         "Utilize todas as fontes de dados disponíveis"
         "para compilar um perfil detalhado,"
         "focando nos principais tomadores de decisão, negócios recentes"
         "desenvolvimentos e necessidades potenciais"
         "que se alinham com nossas ofertas."
         "Esta tarefa é crucial para o ajuste de"
         "nossa estratégia de engajamento de forma eficaz.\n"
         "Não faça suposições e"
         "use apenas informações sobre as quais você tenha certeza absoluta."
    ),
    expected_output=(
        "Um relatório abrangente sobre {lead_name},"
        "incluindo histórico da empresa"
        "pessoal-chave, marcos recentes e necessidades identificadas."
        "Destaque áreas potenciais onde"
        "nossas soluções podem agregar valor"
        "e sugerir estratégias de engajamento personalizadas."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- A tarefa de divulgação personalizada está usando sua ferramenta personalizada `SentimentAnalysisTool`, bem como `SerperDevTool` da tripulaçãoAI (search_tool).

In [12]:
personalized_outreach_task = Task (
     description=(
         "Usando os insights coletados do"
         "relatório de perfil de lead em {lead_name},"
         "criar uma campanha de divulgação personalizada"
         "destinado a {key_decision_maker},"
         "a {position} de {lead_name}. "
         "A campanha deve abordar seu {milestone} recente"
         "e como nossas soluções podem apoiar seus objetivos."
         "Sua comunicação deve ressoar"
         "com a cultura e os valores da empresa {lead_name},"
         "demonstrando uma profunda compreensão de"
         "seus negócios e necessidades.\n"
         "Não faça suposições e apenas"
         "use informações sobre as quais você tem certeza absoluta."
     ),
     expected_output=(
         "Uma série de rascunhos de e-mail personalizados"
         "adaptado para {lead_name}, "
         "visando especificamente {key_decision_maker}."
         "Cada rascunho deve incluir"
         "uma narrativa convincente que conecta nossas soluções"
         "com suas conquistas recentes e objetivos futuros."
         "Certifique-se de que o tom seja envolvente e profissional"
         "e alinhado com a identidade corporativa de {lead_name}."
     ),
     tools=[sentiment_analysis_tool, search_tool],
     agent=lead_sales_rep_agent,
)

## Criando a tripulação Crew

In [13]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Executando a tripulação 
**Observação**: LLMs podem fornecer saídas diferentes para a mesma entrada, portanto, o que você obtém pode ser diferente do que vê no vídeo.

In [14]:
inputs = {
    "lead_name": "Up link distribuidora",
    "industry": "Distribuidora de equipamentos de informática",
    "key_decision_maker": "Elcio",
    "position": "CEO",
    "milestone": "lançamento do produto"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Representante de Vendas
 [INFO]: == Starting Task: Conduza uma análise aprofundada de Up link distribuidora,uma empresa do setor Distribuidora de equipamentos de informáticaque recentemente demonstrou interesse em nossas soluções.Utilize todas as fontes de dados disponíveispara compilar um perfil detalhado,focando nos principais tomadores de decisão, negócios recentesdesenvolvimentos e necessidades potenciaisque se alinham com nossas ofertas.Esta tarefa é crucial para o ajuste denossa estratégia de engajamento de forma eficaz.
Não faça suposições euse apenas informações sobre as quais você tenha certeza absoluta.


> Entering new CrewAgentExecutor chain...
Para começar, preciso pesquisar informações sobre a empresa Up link distribuidora na internet. Quero informações sobre o histórico da empresa, pessoal-chave, marcos recentes e necessidades identificadas.

Action: Search the internet
Action Input: {"search_query": "Up link distribuidora perfil da empresa"} 

- Display the final result as Markdown.

In [15]:
from IPython.display import Markdown
Markdown(result)

Aqui estão três rascunhos de e-mail personalizados para Elcio, CEO da Up link distribuidora:

1. "Olá Elcio, espero que esteja bem. Meu nome é [seu nome] e sou Representante Líder de Vendas na Inovabot. Tenho acompanhado a trajetória da Up link distribuidora e fiquei impressionado com sua dedicação e sucesso. Acreditamos que nossas soluções podem ajudar a Up link distribuidora a melhorar ainda mais suas operações. Gostaria de marcar uma reunião para apresentar nossas soluções e discutir como elas podem beneficiar sua empresa. Aguardo sua resposta. Atenciosamente, [seu nome]"

2. "Caro Elcio, recentemente tomei conhecimento de que a Up link distribuidora está em processo de expansão, especialmente no que diz respeito à equipe de vendas. Na Inovabot, temos várias soluções que podem apoiar essa expansão, ajudando a melhorar a gestão de estoque e logística. Gostaria de explorar essa oportunidade em uma reunião. Aguardo seu retorno. Com os melhores cumprimentos, [seu nome]"

3. "Olá Elcio, tenho notado o desejo da Up link distribuidora de expandir seu alcance de mercado. A Inovabot tem uma variedade de soluções que podem ajudar a aprimorar suas estratégias de vendas e marketing, apoiando a expansão do alcance de mercado da Up link distribuidora. Gostaria de agendar uma reunião para discutir isso mais a fundo. Estou ansioso para ouvir de você. Cordialmente, [seu nome]"